# timeframe: 2011-02-21 to 2018-09-09

In [7]:
import utils
import pandas as pd
import numpy as np

import glob

import joblib

pd.options.display.colheader_justify = 'left'
pd.options.display.column_space = 5
pd.options.display.expand_frame_repr
pd.options.display.max_colwidth = 150

# Concatenate different data files into single CSV and import into pandas

In [8]:
build_master = False
while build_master:
    path = 'data/reddit_api_pulled_csvs/'
    allfiles = glob.glob(path + '*.csv')
    df = pd.DataFrame()
    list_ = []
    for file_ in allfiles:
        temp = pd.read_csv(file_,index_col=None, header=0)
        list_.append(temp)
    df = pd.concat(list_)

    df.drop(columns='Unnamed: 0', inplace=True)

    df.to_csv('data/raw/ten_comments_raw.csv')
    break
# simply import pre-concatenated CSV into a DF
while not build_master:
    path = 'data/raw/ten_comments_raw.csv'
    df = pd.read_csv(path) #, dtype = {'created_utc': np.float64})
    df.drop(columns='Unnamed: 0', inplace=True)
    break

# Clean DataFrame

In [9]:
df.dropna(subset=['score', 'body'], inplace=True)
df.drop(index = df[df.score.str.contains('[a-zA-Z]', regex=True)].index, inplace=True) # drop any string characters that ended up in score

# Convert dtypes
df.score = df.score.astype('int')
df.post_score = df.post_score.astype('int')
df.created_utc = df.created_utc.astype('float')
df['created_utc'] = df['created_utc'].apply(np.int64)
df['created_loc'] = df['created_loc'].apply(np.int64)

# Convert unix to datetime
df.created_loc = pd.to_datetime(df.created_loc,unit='s')
df.created_utc = pd.to_datetime(df.created_utc,unit='s')

# Drop removed and deleted reddit comments
df.body = df.body.drop(index = df[df.body == '[deleted]'].index)
df.body = df.body.drop(index = df[df.body == '[removed]'].index)

# Drop duplicate comments
df.drop_duplicates(subset='comment_id', inplace=True)

utils.string_clean_df_column(df, 'body')

#Drop NAs
df.dropna(subset=['body'],inplace=True)

# Save to Joblib

In [522]:
joblib.dump(df, 'data/clean/clean_df.joblib')

['data/clean/clean_df.joblib']